In [ ]:
import pathlib
import os

import cv2
import pandas as pd 
import numpy as np
import albumentations
import matplotlib.pyplot as plt

PROJECT_ROOT = pathlib.Path('.')

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
DATASET_DIR = DATA_DIR / 'YOLO_DATASET'

### Winter photo example for `classifier`

![](./../data/winter_traffic_signs_example/2_cut.png)
![](./../data/winter_traffic_signs_example/1_cut.png)

In [ ]:
img = cv2.imread(
    str(DATA_DIR / 'STOCK_SIGNS' / '1.8.png')
)
img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)

plt.imshow(img)

In [ ]:
class Winterize(albumentations.ImageOnlyTransform):
    def __init__(self, contrast_limit=0.8, tiles_count=5, tiles_size=5, tiles_relative=True, always_apply=False, p=0.5):
        super(Winterize, self).__init__(always_apply, p)
        self.contrast_limit = contrast_limit
        self.tiles_count = tiles_count
        self.tiles_size = tiles_size,
        self.tiles_relative = tiles_relative
        
    # TODO: implement white noise + gaussian on alpha channel of input image
    # TODO: implement while-gray random cuts from image
    def _transform(self, img):
        self.noise = np.random.normal(1, 1, img.shape)
        img = albumentations.brightness_contrast_adjust(img)
        return self.noise 

    def apply(self, img, clip_limit=2, **params):
        if albumentations.get_num_channels(img) != 4:
            raise TypeError("Winterize transformation expects RGBA image")

        return self._transform(img)

    def get_params(self):
        return {'contrast_limit': self.contrast_limit, 'tiles_count': self.tiles_count, 'tiles_size': self.tiles_size, 'tiles_relation': self.tiles_relative}

    def get_transform_init_args_names(self):
        return ("contrast_limit", "tiles_count", "tiles_size", "tiles_relative")

a = Winterize()

img_t = a.apply(img)
plt.imshow(img_t)

In [ ]:
Winterize.__annotations__

### Winter photo example for `detector`
![](./../data/winter_traffic_signs_example/1_full.png)

In [ ]:
import pathlib
import os

import cv2
import pandas as pd 
import numpy as np
import albumentations


PROJECT_ROOT = pathlib.Path('.')

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
DATASET_DIR = DATA_DIR / 'YOLO_DATASET'

In [ ]:
sample_img = cv2.imread(
    DATA_DIR / 
)

In [ ]:
nb_name